In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchProfile,
        HnswAlgorithmConfiguration,
    )
import pandas as pd
from azure.core.exceptions import ResourceNotFoundError





In [ ]:
ord("A") - ord("a")
admin_key = ""


In [ ]:
import time

In [ ]:
AZURE_SEARCH_SERVICE = ""
service_endpoint = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
index_name = "nice"

credential = AzureKeyCredential(admin_key)

In [ ]:
!pip install --upgrade azure-search-documents


In [ ]:
import time
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="",  # replace with env var or secret manager in real use
    api_version="2024-02-15-preview",
    azure_endpoint="https://genaitcgazuregpt.openai.azure.com",
)

def get_embeddings(text: str):
    """Get embeddings for text using Azure OpenAI"""
    if not text or text.strip() == "":
        return [0.0] * 1536

    try:
        response = client.embeddings.create(
            input=[text],
            model="text-embedding-ada-002"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embeddings for text '{text[:50]}...': {str(e)}")
        return [0.0] * 1536

# Test
start = time.time()
print(get_embeddings("hi"))
end = time.time()
print("Time taken:", end - start)

[-0.030890777707099915, -0.020455405116081238, -0.01946897804737091, -0.04176744818687439, -0.024972207844257355, 0.02434920147061348, -0.01797635667026043, -0.01763889379799366, -0.006564290262758732, -0.016003498807549477, 0.02594565786421299, -0.0070153214037418365, -0.017924439162015915, -0.011811180040240288, 0.011408820748329163, 0.01650969311594963, 0.03888603672385216, 0.0005836636992171407, 0.032240625470876694, -0.008728591725230217, -0.019728563725948334, -0.004876978695392609, -0.009429474361240864, -0.01427725050598383, -0.022895516827702522, 0.0025066309608519077, 0.010039502754807472, -0.011791710741817951, 0.0025715273804962635, -0.025971615687012672, 0.01444598101079464, 0.0007446883828379214, -0.03571908548474312, -0.014965154230594635, -0.0094489436596632, -0.024725601077079773, 0.007002342492341995, -0.021221185103058815, 0.0191185362637043, -0.0056654722429811954, 0.006213848479092121, -0.0007657798123545945, 0.0013733741361647844, -0.014536836184561253, -0.0230253

In [ ]:
def create_index():
    AZURE_SEARCH_SERVICE = ""
    service_endpoint = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
    index_name = "nice"

    credential = AzureKeyCredential(admin_key)
    """Create index with built-in vectorizer for automatic embedding generation"""


    index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

    try:
        from azure.search.documents.indexes.models import AzureOpenAIVectorizer, AzureOpenAIParameters
        VECTORIZER_AVAILABLE = True
    except ImportError:
        VECTORIZER_AVAILABLE = False
        print("Note: AzureOpenAIVectorizer not available in this SDK version. Using JSON configuration instead.")


    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_profile_name="my-vector-profile"
        )
    ]
    
    if VECTORIZER_AVAILABLE:
        # Method 1: Using SDK classes (newer versions)
        vectorizer = AzureOpenAIVectorizer(
            name="my-openai-vectorizer",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri="https://genaitcgazuregpt.openai.azure.com",
                deployment_id="text-embedding-ada-002",
                model_name="text-embedding-ada-002",
                api_key=""
            )
        )
        
        vector_search = VectorSearch(
            profiles=[
                VectorSearchProfile(
                    name="my-vector-profile",
                    algorithm_configuration_name="my-hnsw",
                    vectorizer_name="my-openai-vectorizer"
                )
            ],
            algorithms=[
                HnswAlgorithmConfiguration(name="my-hnsw")
            ],
            vectorizers=[vectorizer]
        )
    else:
        # Method 2: Using dictionary/JSON configuration (older versions)
        vector_search = VectorSearch(
            profiles=[
                VectorSearchProfile(
                    name="my-vector-profile",
                    algorithm_configuration_name="my-hnsw",
                    vectorizer_name="my-openai-vectorizer"
                )
            ],
            algorithms=[
                HnswAlgorithmConfiguration(name="my-hnsw")
            ],
            vectorizers=[
                {
                    "name": "my-openai-vectorizer",
                    "kind": "azureOpenAI",
                    "azureOpenAIParameters": {
                        "resourceUri": "https://genaitcgazuregpt.openai.azure.com",
                        "deploymentId": "text-embedding-ada-002",
                        "modelName": "text-embedding-ada-002",
                        "apiKey": ""
                    }
                }
            ]
        )
    
    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
    
    try:
        # Check if index exists and delete it
        existing_indexes = [idx.name for idx in index_client.list_indexes()]
        if index_name in existing_indexes:
            index_client.delete_index(index_name)
            print(f"Deleted existing index: {index_name}")
            time.sleep(3)
        
        # Create the new index
        result = index_client.create_index(index)
        print(f"Created index: {index_name}")
        time.sleep(5)
        return True
        
    except Exception as e:
        print(f"Error creating index: {e}")
        return False

create_index()

from azure.search.documents import SearchClient
import uuid

# Reuse existing credential and service_endpoint from earlier
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

def add_documents_to_index(texts):
    """Add documents - embeddings will be generated automatically by built-in vectorizer"""
    docs = []
    for text in texts:
        doc_id = str(uuid.uuid4())
        doc = {
            "id": doc_id,
            "content": text,
            # Note: embedding field will be auto-populated by the built-in vectorizer
        }
        docs.append(doc)

    try:
        result = search_client.upload_documents(documents=docs)
        print(f"Successfully uploaded {len(docs)} documents")
        return True
    except Exception as e:
        print(f"Error uploading documents: {e}")
        return False
        
add_documents_to_index(["good", "bad", "ugly", "hi"])


from azure.search.documents import SearchClient

def extract_all_documents():
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    
    try:
        results = search_client.search(search_text="*", top=1000)
        print("\n--- Retrieved Documents ---")

        all_docs = []
        for result in results:
            # If result is a dict (common), use dict access
            if isinstance(result, dict):
                doc = {
                    "id": result.get("id"),
                    "content": result.get("content"),
                    "score": result.get("@search.score")
                }
            else:
                # Fallback if result is a SearchResult object
                doc = {
                    "id": getattr(result, "id", None),
                    "content": getattr(result, "content", None),
                    "score": result.get("@search.score") if "@search.score" in result else None
                }
            print(doc)
            all_docs.append(doc)
        return all_docs

    except Exception as e:
        print(f"Error fetching documents: {e}")
        return []

# Call the function
extract_all_documents()



from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

def search_by_embedding(query_text, k=3):
    """Search documents similar to the query_text using vector search."""
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    
    try:
        embedding = get_embeddings(query_text)
        vector_query = VectorizedQuery(
            vector=embedding,
            k_nearest_neighbors=k,
            fields="embedding"
        )
        
        results = search_client.search(
            vector_queries=[vector_query],
            top=k
        )

        print(f"\n--- Search Results for: '{query_text}' ---")
        all_results = []
        for result in results:
            doc = {
                "id": result["id"] if "id" in result else None,
                "content": result["content"] if "content" in result else None,
                "score": result["@search.score"] if "@search.score" in result else None
            }
            print(doc)
            all_results.append(doc)

        return all_results

    except Exception as e:
        print(f"Search error: {e}")
        return []

# Example usage
search_by_embedding("hi")

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

def search_by_embedding(query_text, k=3):
    """Search documents similar to the query_text using vector search."""
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    
    try:
        embedding = get_embeddings(query_text)
        vector_query = VectorizedQuery(
            vector=embedding,
            k_nearest_neighbors=k,
            fields="embedding"
        )
        
        results = search_client.search(
            vector_queries=[vector_query],
            top=k
        )

        print(f"\n--- Search Results for: '{query_text}' ---")
        all_results = []
        for result in results:
            doc = {
                "id": result["id"] if "id" in result else None,
                "content": result["content"] if "content" in result else None,
                "score": result["@search.score"] if "@search.score" in result else None
            }
            print(doc)
            all_results.append(doc)

        return all_results

    except Exception as e:
        print(f"Search error: {e}")
        return []

# Example usage
search_by_embedding("hi")


Note: AzureOpenAIVectorizer not available in this SDK version. Using JSON configuration instead.
Deleted existing index: nice
Created index: nice
Successfully uploaded 4 documents

--- Retrieved Documents ---
{'id': '5686a8cc-3884-4d16-8ad9-a38a3987e706', 'content': 'bad', 'score': 1.0}
{'id': '6a01fa72-1357-48e6-bffb-0ac02570894a', 'content': 'ugly', 'score': 1.0}
{'id': '16b55980-f963-4130-9738-0af98736cae7', 'content': 'good', 'score': 1.0}
{'id': '3b942f4c-f944-4323-9270-85695cd4f256', 'content': 'hi', 'score': 1.0}

--- Search Results for: 'hi' ---

--- Search Results for: 'hi' ---


[]